In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec
from lark.data import *

1.8.1
0.8.0a0+e4e171a


In [3]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [4]:
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [9]:
cfg = Config(
    use_overlays=False,
    use_noise=True,
    noise_nsr_dbs = [30, 20, 10, 3],
    sites=None, 
    use_neptune=True, 
    log_batch_metrics=False,
    n_epochs=150, 
    bs=32, 
    lr=1e-3, 
    model='resnest50',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [10]:
cfg.training_dataset_size

152448

In [11]:
prep = MixedSig2Spec(cfg)
main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
posp = torch.nn.Linear(in_features=2048, 
                       out_features=len(cfg.labels), bias=True)
main_model.fc = posp
model = torch.nn.Sequential(prep, main_model)
model = model.cuda()

Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [12]:
lrn = Learner("resnest50-2ndary-labels-full", cfg, model)

/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['secondary_labels'] = self.df_meta['secondary_labels'].str.replace("[\[\]',]", '').str.split()
/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:77: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['filename'] = self.df_meta['filename'].str.replace(".ogg", '.wav')


In [13]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-88


  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/4764 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/4764 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-07 10:28:46 epoch:  23 train loss: 0.009860 train f1: 0.635196 valid loss: 0.010636 valid f1: 0.082278


  0%|          | 0/4764 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,301,29792,3038,469,0.146508
2,0.2,243,31157,1673,527,0.180938
3,0.3,193,31787,1043,577,0.192423
4,0.4,164,32173,657,606,0.206160
5,0.5,145,32429,401,625,0.220365
6,0.6,126,32582,248,644,0.220280
7,0.7,103,32658,172,667,0.197129
8,0.8,88,32735,95,682,0.184680
9,0.9,72,32796,34,698,0.164384


In [12]:
lrn.load_checkpoint('best')

{'epoch': 2,
 'valid_loss': 0.0897514734809336,
 'valid_score': 0.32969215512275696}

In [13]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,461,30369,2461,309,0.249729
2,0.2,319,32325,505,451,0.400251
3,0.3,237,32620,210,533,0.389482
4,0.4,196,32714,116,574,0.362292
5,0.5,166,32759,71,604,0.329692
6,0.6,136,32789,41,634,0.287223
7,0.7,110,32804,26,660,0.242826
8,0.8,77,32814,16,693,0.178447
9,0.9,43,32826,4,727,0.105263
